In [1]:
import transformers
from transformers import CLIPConfig, CLIPModel, CLIPProcessor, CLIPImageProcessor, CLIPTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from pathlib import Path
from gensim.models import Word2Vec
import re

In [10]:
mat_contents = sio.loadmat('subject_1.mat')
words = mat_contents["words"][0]
list_words = []
run = []
length = 4

scan = []
for x in words:

    #print(x[2][0][0])
    if x[2][0][0] < 1:
        word = x[0][0][0][0]
        print(word)
        while('"' in word):
            word = word.replace('"','')
            scan.append(word)
        if len(scan) == length:
            run.append(" ".join(scan))
            scan = []
    else:
        if len(scan) != 0:
            run.append(" ".join(scan))
            scan = []
        list_words.append(run)
        run = []
        scan = []
        

        

Harry
had
never
believed
he
would
meet
a
boy
he
hated
more
than
Dudley,
but
that
was
before
he
met
Draco
Malfoy.
Still,
first-year
Gryffindors
only
had
Potions
with
the
Slytherins,
so
they
didn't
have
to
put
up
with
Malfoy
much.
Or
at
least,
they
didn't
until
they
spotted
a
notice
pinned
up
in
the
Gryffindor
common
room
that
made
them
all
groan.
Flying
lessons
would
be
starting
on
Thursday
--
and
Gryffindor
and
Slytherin
would
be
learning
together.
+
"Typical,"
said
Harry
darkly.
"Just
what
I
always
wanted.
To
make
a
fool
of
myself
on
a
broomstick
in
front
of
Malfoy."
+
He
had
been
looking
forward
to
learning
to
fly
more
than
anything
else.
"You
don't
know
that
you'll
make
a
fool
of
yourself,"
said
Ron
reasonably.
"Anyway,
I
know
Malfoy's
always
going
on
about
how
good
he
is
at
Quidditch,
but
I
bet
that's
all
talk."
+
Malfoy
certainly
did
talk
about
flying
a
lot.
He
complained
loudly
about
first
years
never
getting
on
the
House
Quidditch
teams
and
told
long,
boastful
stories
that
alway

bags
in
ten
minutes.
What
would
the
Dursleys
say
when
he
turned
up
on
the
doorstep?
+
Up
the
front
steps,
up
the
marble
staircase
inside,
and
still
Professor
McGonagall
didn't
say
a
word
to
him.
She
wrenched
open
doors
and
marched
along
corridors
with
Harry
trotting
miserably
behind
her.
Maybe
she
was
taking
him
to
Dumbledore.
He
thought
of
Hagrid,
expelled
but
allowed
to
stay
on
as
gamekeeper.
Perhaps
he
could
be
Hagrid's
assistant.
His
stomach
twisted
as
he
imagined
it,
watching
Ron
and
the
others
becoming
wizards
while
he
stumped
around
the
grounds
carrying
Hagrid's
bag.
+
Professor
McGonagall
stopped
outside
a
classroom.
She
opened
the
door
and
poked
her
head
inside.
+
"Excuse
me,
Professor
Flitwick,
could
I
borrow
Wood
for
a
moment?"
+
Wood?
thought
Harry,
bewildered;
was
Wood
a
cane
she
was
going
to
use
on
him?
But
Wood
turned
out
to
be
a
person,
a
burly
fifth-year
boy
who
came
out
of
Flitwick's
class
looking
confused.
+
"Follow
me,
you
two,"
said
Professor
McGonagall,
and
they
m

remember
what
I
said
when
you're
on
the
train
home
tomorrow,
you're
so
—"
+
But
what
they
were,
they
didn't
find
out.
Hermione
had
turned
to
the
portrait
of
the
Fat
Lady
to
get
back
inside
and
found
herself
facing
an
empty
painting.
The
Fat
Lady
had
gone
on
a
nighttime
visit
and
Hermione
was
locked
out
of
Gryffindor
Tower.
+
"Now
what
am
I
going
to
do?"
she
asked
shrilly.
+
"That's
your
problem,"
said
Ron.
"We've
got
to
go,
we're
going
to
be
late."
+
They
hadn't
even
reached
the
end
of
the
corridor
when
Hermione
caught
up
with
them.
+
"I'm
coming
with
you,"
she
said.
"You
are
@not."
+
"D'you
think
I'm
going
to
stand
out
here
and
wait
for
Filch
to
catch
me?
If
he
finds
all
three
of
us
I'll
tell
him
the
truth,
that
I
was
trying
to
stop
you,
and
you
can
back
me
up."
+
"You've
got
some
nerve
—"
said
Ron
loudly.
+
"Shut
up,
both
of
you!"
said
Harry
sharply.
"I
heard
something."
+
It
was
a
sort
of
snuffling.
+
"Mrs.
Norris?"
breathed
Ron,
squinting
through
the
dark.
+
It
wasn't
Mrs.
Norris.


In [12]:
print(len(list_words))
print(list_words)

4
[['Typical, Just Malfoy. You', "yourself, Anyway, talk. It's", 'Remembrall! Gran … …', "… What's on? Malfoy's", 'Professor. Just looking, Well,', 'for? Everyone up. Stick', "broom, and ‘Up!' UP!", 'Now, hard, Keep —', 'Come boy! Broken wrist,', 'Come get. None dear.', 'Did lump? Shut Malfoy,', 'Ooh, Longbottom? Never Parvati.', "Look! It's him."], ['Give Malfoy, I tree?', 'Give @here! Come Potter!', '@No! Madam trouble. Give', 'here, or broom! Oh,', 'yeah? No Malfoy, Catch', 'then! HARRY POTTER! @Never', '— — — It', '— Be — But', "— That's now. Excuse", 'moment? Follow two, In', 'here. Out, Peeves! Potter,', 'Seeker. Are Professor? Absolutely,', 'The Potter? He dive,', "Didn't it. Ever Potter?", "Wood's team, He's too,", 'Light say. I ….', 'I you. Your proud,', "He himself. You're @joking.", '@Seeker? But — —', 'century, Wood me. I', 'week, Only secret. Well', 'done, Wood Beaters. I', 'year, We us. Anyway,', 'school. Bet you.'], ["Having Muggles? You're you,", "I'd own, Tonight, supp

In [57]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

inputs = processor(text=list_words[0], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
# text_embeds is in the same space as the given image
text_embeds = outputs['text_embeds']

# pooler_output is just the text embedding 
# This is what we want to use I think 
pooler_output = outputs['text_model_output']['pooler_output']

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [60]:
print(pooler_output)
print(text_embeds)

tensor([[-0.7876, -1.6431,  1.6882,  ...,  0.2024, -0.9249, -0.7751],
        [ 0.2186, -1.0752,  0.0877,  ..., -1.6650,  0.8878, -1.0070],
        [ 0.2151, -0.9554,  1.6817,  ...,  0.4437,  0.3204,  0.3724],
        ...,
        [-0.7533, -1.1989, -0.1625,  ..., -0.7693, -1.1238, -0.2306],
        [-0.5395,  1.6301,  0.5859,  ..., -1.2589, -1.1547, -0.2034],
        [ 0.5473,  0.7523,  0.1416,  ..., -0.4999, -0.5300,  0.9937]],
       grad_fn=<IndexBackward0>)
tensor([[ 0.0181,  0.0116, -0.0054,  ..., -0.0141, -0.0253,  0.0194],
        [ 0.0138, -0.0112,  0.0184,  ...,  0.0176, -0.0231,  0.0341],
        [ 0.0290,  0.0164,  0.0088,  ...,  0.0096, -0.0047, -0.0099],
        ...,
        [ 0.0365, -0.0249,  0.0473,  ...,  0.0166,  0.0396,  0.0049],
        [ 0.0096,  0.0085,  0.0144,  ..., -0.0061,  0.0084, -0.0091],
        [-0.0115, -0.0300,  0.0041,  ...,  0.0088, -0.0150, -0.0295]],
       grad_fn=<DivBackward0>)
